In [23]:
import pandas as pd
import re
import glob 
import os

import numpy as np
import scipy.signal as signal
import pywt
from scipy.fft import fft
from scipy.stats import entropy
import pyedflib
import warnings
warnings.filterwarnings("ignore")

# Butterworth band-pass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    """Creates a Butterworth bandpass filter with the given cutoff frequencies."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def apply_butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Applies a Butterworth bandpass filter to the data."""
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

# Discrete Wavelet Transform (DWT) for feature extraction
def dwt_features(data, wavelet='db4', level=5):
    """Applies Discrete Wavelet Transform and extracts features from the coefficients."""
    coeffs = pywt.wavedec(data, wavelet, level=level)
    # Extract energy from each level of DWT coefficients
    features = [np.sum(np.square(c)) for c in coeffs]
    return features

# Spectral features (e.g., power spectral density)
def spectral_features(data, fs):
    """Calculates spectral features such as spectral power."""
    freqs, psd = signal.welch(data, fs)
    # Example: Extracting power in different frequency bands
    delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
    theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
    alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
    beta_power = np.trapz(psd[(freqs >= 13) & (freqs <= 30)])
    return [delta_power, theta_power, alpha_power, beta_power]

# Hjorth Parameters
def hjorth_parameters(data):
    """Calculates Hjorth parameters: activity, mobility, and complexity."""
    activity = np.var(data)
    mobility = np.sqrt(np.var(np.diff(data)) / activity)
    complexity = np.sqrt(np.var(np.diff(np.diff(data))) / np.var(np.diff(data)))
    return [activity, mobility, complexity]

# Entropy measures
def entropy_features(data):
    """Calculates entropy measures (e.g., Shannon entropy)."""
    hist, _ = np.histogram(data, bins=256, density=True)
    shannon_entropy = entropy(hist, base=2)  # Shannon Entropy
    return [shannon_entropy]

def extract_features(data, fs):
    """Extracts multiple features from the input data and returns feature names along with values."""
    # Apply bandpass filtering
    filtered_data = apply_butter_bandpass_filter(data, 0.5, 30, fs)
    
    # Extract DWT features
    dwt_feats = dwt_features(filtered_data)
    dwt_feat_names = [f'DWT_Energy_Level_{i+1}' for i in range(len(dwt_feats))]

    # Extract spectral features
    spectral_feats = spectral_features(filtered_data, fs)
    spectral_feat_names = ['Delta_Power', 'Theta_Power', 'Alpha_Power', 'Beta_Power']

    # Calculate Hjorth parameters
    hjorth_feats = hjorth_parameters(filtered_data)
    hjorth_feat_names = ['Hjorth_Activity', 'Hjorth_Mobility', 'Hjorth_Complexity']

    # Calculate entropy features
    entropy_feats = entropy_features(filtered_data)
    entropy_feat_names = ['Shannon_Entropy']

    # Concatenate all features and names into single lists
    all_features = dwt_feats + spectral_feats + hjorth_feats + entropy_feats
    all_feature_names = dwt_feat_names + spectral_feat_names + hjorth_feat_names + entropy_feat_names
    
    return all_features, all_feature_names



# Function to convert time strings to seconds
def time_to_seconds(t):
    try:
        if pd.isnull(t) or t == '':
            return None
        parts = list(map(int, re.split('[:]', str(t))))
        if len(parts) == 3:
            return parts[0] * 3600 + parts[1] * 60 + parts[2]
        elif len(parts) == 2:
            return parts[0] * 60 + parts[1]
        elif len(parts) == 1:
            return parts[0]
        else:
            return None
    except ValueError:
        return None

def parse_intervals(file_path):
    # Extract metadata from filename
    filename_id = file_path.split('/')[-1].split('_')

    id_match = filename_id[0]
    age_match = filename_id[1]
    exp_match = filename_id[2]
    
    # Assign extracted values or None if not found
    
    # Read the file into a DataFrame
    intervals_df = pd.read_csv(file_path, sep='\t', header=None, names=['NN', 'Время', 'Маркер'])
    intervals_df['Начало'] = intervals_df['Время'].apply(time_to_seconds)

    # Ensure 'Начало' column is numeric, removing non-numeric values
    intervals_df['Начало'] = pd.to_numeric(intervals_df['Начало'], errors='coerce')
    intervals_df = intervals_df.dropna(subset=['Начало']).reset_index(drop=True)

    paired_intervals = []
    open_markers = {}

    for _, row in intervals_df.iterrows():
        marker_base = re.match(r"([a-zA-Z]+)", row['Маркер']).group(0)
        
        if marker_base in open_markers:
            start_row = open_markers[marker_base]
            if isinstance(row['Начало'], (int, float)) and isinstance(start_row['Начало'], (int, float)):
                if row['Начало'] > start_row['Начало']:
                    paired_intervals.append((start_row['Начало'], row['Начало'], marker_base))
            del open_markers[marker_base]
        else:
            open_markers[marker_base] = row

    # Create a DataFrame with valid pairs
    if paired_intervals:
        valid_intervals_df = pd.DataFrame(
            paired_intervals,
            columns=['X_FROM', 'X_TO', 'LABEL']
        )
        # Add extracted metadata as new columns
        valid_intervals_df['ID'] = id_match
        valid_intervals_df['AGE_MONTHS'] = age_match
        valid_intervals_df['EXPERIMENT_TYPE'] = exp_match
    else:
        valid_intervals_df = pd.DataFrame(columns=['X_FROM', 'X_TO', 'LABEL', 'ID', 'AGE_MONTHS', 'EXPERIMENT_TYPE'])

    return valid_intervals_df


# Function to process all .txt files in a folder
def process_folder(folder_path):
    all_intervals = []
    
    for file_path in glob.glob(os.path.join(folder_path, "*.txt")):
        valid_intervals = parse_intervals(file_path)
        
        with pyedflib.EdfReader(file_path[:-4]+".edf") as edf_reader:
            print(file_path[:-4]+".edf")
            n_channels = edf_reader.signals_in_file
            signal_labels = edf_reader.getSignalLabels()
            # Specify the desired channel
            feature_rows = []

            for types in signal_labels:
                channel_index = signal_labels.index(types)
                data = edf_reader.readSignal(channel_index)
                sfreq = edf_reader.getSampleFrequency(channel_index)
                total_duration = len(data) / sfreq

                for _, row in valid_intervals.iterrows():
                    start_time = row['X_FROM']
                    duration = row['X_TO'] - row['X_FROM']
                    end_time = start_time + duration

                    if end_time > total_duration:
                        end_time = total_duration
                        duration = end_time - start_time

                    # Convert times to indices
                    start_idx = int(start_time * sfreq)
                    end_idx = int(end_time * sfreq)
                    # Extract the signal segment
                    signal_data = data[start_idx:end_idx]
                    # print(signal_data)
                    # Extract features from this segment
                    features, feature_names = extract_features(signal_data, 256)
                    # Convert features to DataFrame row and transpose
                    feature_row = pd.DataFrame([features]).T
                    # print(feature_row)

                    feature_row = feature_row.transpose()
                    feature_row.columns = [f"{i}_{types}" for i in feature_names]

                    # Append the feature row
                    feature_rows.append(feature_row)

                # Concatenate all feature rows with the original DataFrame
            features_df = pd.concat(feature_rows, ignore_index=True)
            df_with_features = pd.concat([valid_intervals.reset_index(drop=True), features_df], axis=1)

            all_intervals.append(df_with_features)
    # Concatenate all DataFrames
    all_intervals_df = pd.concat(all_intervals, ignore_index=True)
    return all_intervals_df

# Example usage
folder_path = '/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)'  # Replace with your folder path
all_valid_intervals = process_folder(folder_path)
all_valid_intervals.fillna(0)



/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x3_9m_Xyl01(Pharm!)_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_BL_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_Dex003(Pharm!)_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_H2O_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x6_14m_BL_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x3_12m_BL_6h.edf


,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,DWT_Energy_Level_1_FrL,DWT_Energy_Level_2_FrL,DWT_Energy_Level_3_FrL,DWT_Energy_Level_4_FrL,DWT_Energy_Level_5_FrL,DWT_Energy_Level_6_FrL,Delta_Power_FrL,Theta_Power_FrL,Alpha_Power_FrL,Beta_Power_FrL,Hjorth_Activity_FrL,Hjorth_Mobility_FrL,Hjorth_Complexity_FrL,Shannon_Entropy_FrL,DWT_Energy_Level_1_FrR,DWT_Energy_Level_2_FrR,DWT_Energy_Level_3_FrR,DWT_Energy_Level_4_FrR,DWT_Energy_Level_5_FrR,DWT_Energy_Level_6_FrR,Delta_Power_FrR,Theta_Power_FrR,Alpha_Power_FrR,Beta_Power_FrR,Hjorth_Activity_FrR,Hjorth_Mobility_FrR,Hjorth_Complexity_FrR,Shannon_Entropy_FrR,DWT_Energy_Level_1_OcR,DWT_Energy_Level_2_OcR,DWT_Energy_Level_3_OcR,DWT_Energy_Level_4_OcR,DWT_Energy_Level_5_OcR,DWT_Energy_Level_6_OcR,Delta_Power_OcR,Theta_Power_OcR,Alpha_Power_OcR,Beta_Power_OcR,Hjorth_Activity_OcR,Hjorth_Mobility_OcR,Hjorth_Complexity_OcR,Shannon_Entropy_OcR
0,65.0,74.0,swd,Ati4x3,9m,Xyl01(Pharm!),41.736788,115.921150,80.741924,22.575744,1.675793,0.019530,0.009218,0.030985,0.022560,0.011157,0.072795,0.228740,0.379115,7.212391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,86.0,97.0,swd,Ati4x3,9m,Xyl01(Pharm!),78.885679,172.981258,106.916182,28.074434,2.159237,0.028285,0.015618,0.038948,0.023551,0.010889,0.088302,0.214750,0.379114,7.105269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,102.0,106.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.980034,55.922019,35.043307,10.074958,0.542615,0.007490,0.012359,0.037051,0.023528,0.010274,0.078356,0.218989,0.364777,7.191623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,111.0,115.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.685810,41.485386,28.798218,8.672198,0.756478,0.011179,0.014575,0.027515,0.019518,0.009641,0.064530,0.230026,0.402949,7.055385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,117.0,121.0,swd,Ati4x3,9m,Xyl01(Pharm!),22.536150,39.467002,28.776488,3.758832,0.377413,0.005117,0.010377,0.028629,0.016348,0.005721,0.057521,0.208492,0.357971,6.950334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,0.0,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.029268,10.402153,6.728154,6.698533,1.240794,0.027319,0.001429,0.003000,0.001667,0.002380,0.008750,0.333357,0.579698,6.883252
1706,0.0,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.027771,8.403652,5.854089,4.650918,0.939232,0.016625,0.001235,0.002197,0.001125,0.001588,0.006595,0.311715,0.556077,6.609950
1707,0.0,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.192633,4.123271,4.130203,4.997128,0.993607,0.021221,0.001237,0.001518,0.001471,0.003436,0.008159,0.395608,0.598337,6.939620
1708,0.0,0.0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,

In [35]:
def process_folder(folder_path):
    all_intervals = []
    
    for file_path in glob.glob(os.path.join(folder_path, "*.txt")):
        valid_intervals = parse_intervals(file_path)
        
        with pyedflib.EdfReader(file_path[:-4]+".edf") as edf_reader:
            print(file_path[:-4]+".edf")
            n_channels = edf_reader.signals_in_file
            signal_labels = edf_reader.getSignalLabels()
            # Specify the desired channel
            feature_rows = []

            for types in signal_labels:
                channel_index = signal_labels.index(types)
                data = edf_reader.readSignal(channel_index)
                sfreq = edf_reader.getSampleFrequency(channel_index)
                total_duration = len(data) / sfreq

                for _, row in valid_intervals.iterrows():
                    start_time = row['X_FROM']
                    end_time = row['X_TO']

                    if end_time > total_duration:
                        end_time = total_duration

                    # Convert times to indices
                    start_idx = int(start_time * sfreq)
                    end_idx = int(end_time * sfreq)
                    # Extract the signal segment
                    signal_data = data[start_idx:end_idx]
                    # print(signal_data)
                    # Extract features from this segment
                    features, feature_names = extract_features(signal_data, 256)
                    # Convert features to DataFrame row and transpose
                    feature_row = pd.DataFrame([features]).T
                    # print(feature_row)

                    feature_row = feature_row.transpose()
                    feature_row.columns = [f"{i}_{types}" for i in feature_names]

                    # Append the feature row
                    feature_rows.append(feature_row)

                # Concatenate all feature rows with the original DataFrame
            features_df = pd.concat(feature_rows, ignore_index=True)
            df_with_features = pd.concat([valid_intervals.reset_index(drop=True), features_df], axis=1)

            all_intervals.append(df_with_features)
    # Concatenate all DataFrames
    all_intervals_df = pd.concat(all_intervals, ignore_index=True)
    return all_intervals_df

# Example usage
folder_path = '/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)'  # Replace with your folder path
all_valid_intervals = process_folder(folder_path)
all_valid_intervals = all_valid_intervals.dropna(subset=['ID', 'LABEL'])
all_valid_intervals.fillna(0)

/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x3_9m_Xyl01(Pharm!)_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_BL_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_Dex003(Pharm!)_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_H2O_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x6_14m_BL_6h.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x3_12m_BL_6h.edf


,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,DWT_Energy_Level_1_FrL,DWT_Energy_Level_2_FrL,DWT_Energy_Level_3_FrL,DWT_Energy_Level_4_FrL,DWT_Energy_Level_5_FrL,DWT_Energy_Level_6_FrL,Delta_Power_FrL,Theta_Power_FrL,Alpha_Power_FrL,Beta_Power_FrL,Hjorth_Activity_FrL,Hjorth_Mobility_FrL,Hjorth_Complexity_FrL,Shannon_Entropy_FrL,DWT_Energy_Level_1_FrR,DWT_Energy_Level_2_FrR,DWT_Energy_Level_3_FrR,DWT_Energy_Level_4_FrR,DWT_Energy_Level_5_FrR,DWT_Energy_Level_6_FrR,Delta_Power_FrR,Theta_Power_FrR,Alpha_Power_FrR,Beta_Power_FrR,Hjorth_Activity_FrR,Hjorth_Mobility_FrR,Hjorth_Complexity_FrR,Shannon_Entropy_FrR,DWT_Energy_Level_1_OcR,DWT_Energy_Level_2_OcR,DWT_Energy_Level_3_OcR,DWT_Energy_Level_4_OcR,DWT_Energy_Level_5_OcR,DWT_Energy_Level_6_OcR,Delta_Power_OcR,Theta_Power_OcR,Alpha_Power_OcR,Beta_Power_OcR,Hjorth_Activity_OcR,Hjorth_Mobility_OcR,Hjorth_Complexity_OcR,Shannon_Entropy_OcR
0,65.0,74.0,swd,Ati4x3,9m,Xyl01(Pharm!),41.736788,115.921150,80.741924,22.575744,1.675793,0.019530,0.009218,0.030985,0.022560,0.011157,0.072795,0.228740,0.379115,7.212391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,86.0,97.0,swd,Ati4x3,9m,Xyl01(Pharm!),78.885679,172.981258,106.916182,28.074434,2.159237,0.028285,0.015618,0.038948,0.023551,0.010889,0.088302,0.214750,0.379114,7.105269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,102.0,106.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.980034,55.922019,35.043307,10.074958,0.542615,0.007490,0.012359,0.037051,0.023528,0.010274,0.078356,0.218989,0.364777,7.191623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,111.0,115.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.685810,41.485386,28.798218,8.672198,0.756478,0.011179,0.014575,0.027515,0.019518,0.009641,0.064530,0.230026,0.402949,7.055385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,117.0,121.0,swd,Ati4x3,9m,Xyl01(Pharm!),22.536150,39.467002,28.776488,3.758832,0.377413,0.005117,0.010377,0.028629,0.016348,0.005721,0.057521,0.208492,0.357971,6.950334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,20080.0,20089.0,swd,Ati4x3,12m,BL,51.875772,148.515264,110.284638,33.662867,2.371608,0.030864,0.013946,0.039588,0.026866,0.015265,0.095078,0.233905,0.393691,6.988816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1526,20605.0,20614.0,swd,Ati4x3,12m,BL,54.413026,144.923011,103.074536,42.541546,2.882179,0.039337,0.014793,0.038007,0.026062,0.016676,0.096581,0.244714,0.409585,6.945456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1527,20631.0,20636.0,swd,Ati4x3,12m,BL,30.200842,68.234215,50.315813,21.313092,1.455576,0.021546,0.011645,0.034554,0.024760,0.014584,0.085044,0.244717,0.410274,6.900906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1528,20644.0,20649.0,swd,Ati4x3,12m,BL,26.370483,65.663622,65.724688,20.621628,1.381368,0.016610,0.009557,0.041003,0.027432,0.016452,0.089523,0.242957,0.393722,6.998210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
1. EDA -> in (folder: txt+ edf) -> out: dataframe (aka csv)
2. MODEL : in: csv -> csv (+1 MARKER)
3. in: folder (edf) -> out folder (edf + txt)

FINAL : predict (edf) -> return csv (+1 marker)

In [36]:
all_valid_intervals.to_csv("train.csv", index=False)


In [37]:
folder_path = '/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard'  # Replace with your folder path
all_valid_intervals = process_folder(folder_path)
all_valid_intervals.to_csv("validation.csv", index=False)


/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4y3_12m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati5x1_10m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4y2_11m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati5x1_11m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4x3B_15m_H2O_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4x1_15m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4x1_15m_H2O_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_golden_standard/Ati4x3_12m_BL_6h_edited.edf
/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/EC

In [33]:
all_valid_intervals.to_csv("validation.csv", index=False)


In [39]:
all_valid_intervals[all_valid_intervals.ID=='Ati4y3']

,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,DWT_Energy_Level_1_FrL,DWT_Energy_Level_2_FrL,DWT_Energy_Level_3_FrL,DWT_Energy_Level_4_FrL,DWT_Energy_Level_5_FrL,DWT_Energy_Level_6_FrL,Delta_Power_FrL,Theta_Power_FrL,Alpha_Power_FrL,Beta_Power_FrL,Hjorth_Activity_FrL,Hjorth_Mobility_FrL,Hjorth_Complexity_FrL,Shannon_Entropy_FrL,DWT_Energy_Level_1_FrR,DWT_Energy_Level_2_FrR,DWT_Energy_Level_3_FrR,DWT_Energy_Level_4_FrR,DWT_Energy_Level_5_FrR,DWT_Energy_Level_6_FrR,Delta_Power_FrR,Theta_Power_FrR,Alpha_Power_FrR,Beta_Power_FrR,Hjorth_Activity_FrR,Hjorth_Mobility_FrR,Hjorth_Complexity_FrR,Shannon_Entropy_FrR,DWT_Energy_Level_1_OcR,DWT_Energy_Level_2_OcR,DWT_Energy_Level_3_OcR,DWT_Energy_Level_4_OcR,DWT_Energy_Level_5_OcR,DWT_Energy_Level_6_OcR,Delta_Power_OcR,Theta_Power_OcR,Alpha_Power_OcR,Beta_Power_OcR,Hjorth_Activity_OcR,Hjorth_Mobility_OcR,Hjorth_Complexity_OcR,Shannon_Entropy_OcR
0,0.0,20.0,EMPTY,Ati4y3,12m,BL,40.243653,14.290934,9.893603,2.401972,0.375553,0.005751,0.003571,0.001810,0.000822,0.000419,0.008340,0.162210,0.429255,6.966838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40.0,60.0,EMPTY,Ati4y3,12m,BL,77.330705,29.150397,19.905851,4.116761,0.377041,0.005938,0.007011,0.003675,0.002170,0.000616,0.016285,0.148648,0.372563,7.060856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80.0,100.0,EMPTY,Ati4y3,12m,BL,13.741715,11.357756,7.178042,2.873398,0.444803,0.006791,0.001237,0.001532,0.000732,0.000492,0.004439,0.216179,0.469153,6.735683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120.0,140.0,EMPTY,Ati4y3,12m,BL,72.859692,30.468718,13.177216,3.122007,0.323350,0.005816,0.006211,0.003861,0.001501,0.000521,0.014468,0.147131,0.378575,6.998132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,160.0,180.0,EMPTY,Ati4y3,12m,BL,5.889452,4.516121,2.792503,2.951454,0.562337,0.011200,0.000573,0.000563,0.000218,0.000493,0.002082,0.294811,0.571759,7.046930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,21400.0,21420.0,EMPTY,Ati4y3,12m,BL,7.462269,6.009376,2.476960,2.289426,0.463183,0.009530,0.000657,0.000800,0.000194,0.000372,0.002289,0.260170,0.567598,7.246655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,21440.0,21460.0,EMPTY,Ati4y3,12m,BL,8.883669,6.270691,2.870276,2.504970,0.462039,0.009578,0.000723,0.000827,0.000211,0.000442,0.002622,0.253157,0.557343,7.255389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,21480.0,21500.0,EMPTY,Ati4y3,12m,BL,7.362344,4.314280,3.347370,2.714325,0.498668,0.009354,0.000677,0.000623,0.000261,0.000455,0.002267,0.276522,0.550508,7.274205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,21520.0,21540.0,EMPTY,Ati4y3,12m,BL,8.033753,4.524707,3.326098,2.475582,0.387534,0.006673,0.000749,0.000518,0.000287,0.000423,0.002265,0.257887,0.527836,6.957689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
all_valid_intervals.to_csv("train.csv", index=False)
all_valid_intervals

,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,DWT_Energy_Level_1_FrL,DWT_Energy_Level_2_FrL,DWT_Energy_Level_3_FrL,DWT_Energy_Level_4_FrL,DWT_Energy_Level_5_FrL,DWT_Energy_Level_6_FrL,Delta_Power_FrL,Theta_Power_FrL,Alpha_Power_FrL,Beta_Power_FrL,Hjorth_Activity_FrL,Hjorth_Mobility_FrL,Hjorth_Complexity_FrL,Shannon_Entropy_FrL,DWT_Energy_Level_1_FrR,DWT_Energy_Level_2_FrR,DWT_Energy_Level_3_FrR,DWT_Energy_Level_4_FrR,DWT_Energy_Level_5_FrR,DWT_Energy_Level_6_FrR,Delta_Power_FrR,Theta_Power_FrR,Alpha_Power_FrR,Beta_Power_FrR,Hjorth_Activity_FrR,Hjorth_Mobility_FrR,Hjorth_Complexity_FrR,Shannon_Entropy_FrR,DWT_Energy_Level_1_OcR,DWT_Energy_Level_2_OcR,DWT_Energy_Level_3_OcR,DWT_Energy_Level_4_OcR,DWT_Energy_Level_5_OcR,DWT_Energy_Level_6_OcR,Delta_Power_OcR,Theta_Power_OcR,Alpha_Power_OcR,Beta_Power_OcR,Hjorth_Activity_OcR,Hjorth_Mobility_OcR,Hjorth_Complexity_OcR,Shannon_Entropy_OcR
0,65.0,74.0,swd,Ati4x3,9m,Xyl01(Pharm!),41.736788,115.921150,80.741924,22.575744,1.675793,0.019530,0.009218,0.030985,0.022560,0.011157,0.072795,0.228740,0.379115,7.212391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86.0,97.0,swd,Ati4x3,9m,Xyl01(Pharm!),78.885679,172.981258,106.916182,28.074434,2.159237,0.028285,0.015618,0.038948,0.023551,0.010889,0.088302,0.214750,0.379114,7.105269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,102.0,106.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.980034,55.922019,35.043307,10.074958,0.542615,0.007490,0.012359,0.037051,0.023528,0.010274,0.078356,0.218989,0.364777,7.191623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111.0,115.0,swd,Ati4x3,9m,Xyl01(Pharm!),23.685810,41.485386,28.798218,8.672198,0.756478,0.011179,0.014575,0.027515,0.019518,0.009641,0.064530,0.230026,0.402949,7.055385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117.0,121.0,swd,Ati4x3,9m,Xyl01(Pharm!),22.536150,39.467002,28.776488,3.758832,0.377413,0.005117,0.010377,0.028629,0.016348,0.005721,0.057521,0.208492,0.357971,6.950334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.029268,10.402153,6.728154,6.698533,1.240794,0.027319,0.001429,0.003000,0.001667,0.002380,0.008750,0.333357,0.579698,6.883252
1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.027771,8.403652,5.854089,4.650918,0.939232,0.016625,0.001235,0.002197,0.001125,0.001588,0.006595,0.311715,0.556077,6.609950
1707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.192633,4.123271,4.130203,4.997128,0.993607,0.021221,0.001237,0.001518,0.001471,0.003436,0.008159,0.395608,0.598337,6.939620
1708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.046986,4.837281,3.024663,3.520638,0.612678,0.015445,0.001122,0.002302,0.001189,0.002348,0.007447,0.347378,0.597102,6.855550


In [ ]:
folder_path = '/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_unmarked'  # Replace with your folder path
all_valid_intervals = process_folder(folder_path)
all_valid_intervals.to_csv("test.csv", index=False)


In [5]:
pd.set_option('display.max_columns', 200)
all_valid_intervals

,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,DWT_Energy_Level_1_FrL,DWT_Energy_Level_2_FrL,DWT_Energy_Level_3_FrL,DWT_Energy_Level_4_FrL,DWT_Energy_Level_5_FrL,DWT_Energy_Level_6_FrL,Delta_Power_FrL,Theta_Power_FrL,Alpha_Power_FrL,Beta_Power_FrL,Hjorth_Activity_FrL,Hjorth_Mobility_FrL,Hjorth_Complexity_FrL,Shannon_Entropy_FrL,DWT_Energy_Level_1_FrR,DWT_Energy_Level_2_FrR,DWT_Energy_Level_3_FrR,DWT_Energy_Level_4_FrR,DWT_Energy_Level_5_FrR,DWT_Energy_Level_6_FrR,Delta_Power_FrR,Theta_Power_FrR,Alpha_Power_FrR,Beta_Power_FrR,Hjorth_Activity_FrR,Hjorth_Mobility_FrR,Hjorth_Complexity_FrR,Shannon_Entropy_FrR,DWT_Energy_Level_1_OcR,DWT_Energy_Level_2_OcR,DWT_Energy_Level_3_OcR,DWT_Energy_Level_4_OcR,DWT_Energy_Level_5_OcR,DWT_Energy_Level_6_OcR,Delta_Power_OcR,Theta_Power_OcR,Alpha_Power_OcR,Beta_Power_OcR,Hjorth_Activity_OcR,Hjorth_Mobility_OcR,Hjorth_Complexity_OcR,Shannon_Entropy_OcR
0,65.0,74.0,swd,Ati4x3,9m,Xyl01(Pharm!),39.331433,113.053905,82.485687,16.693492,0.403850,0.002910,0.000580,0.014309,0.003237,0.014663,0.069938,0.204877,0.299375,7.254768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86.0,97.0,swd,Ati4x3,9m,Xyl01(Pharm!),82.862672,172.410383,100.283955,19.894257,0.509804,0.003692,0.000878,0.021526,0.003655,0.014503,0.085221,0.191701,0.293015,7.121719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,102.0,106.0,swd,Ati4x3,9m,Xyl01(Pharm!),21.354401,51.524006,41.635955,7.267103,0.175960,0.001221,0.000451,0.018361,0.004197,0.015394,0.076137,0.199441,0.296683,7.275843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111.0,115.0,swd,Ati4x3,9m,Xyl01(Pharm!),24.983978,41.978873,24.868803,6.468560,0.162066,0.001204,0.000683,0.016210,0.004025,0.011476,0.061481,0.200974,0.301110,7.073731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117.0,121.0,swd,Ati4x3,9m,Xyl01(Pharm!),19.001242,39.306772,25.434852,6.024924,0.101470,0.000720,0.000914,0.012457,0.003580,0.010759,0.055985,0.190367,0.284076,7.140839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.781984,10.241250,6.768054,2.190044,0.107689,0.001041,0.000105,0.001679,0.000346,0.001233,0.007095,0.213999,0.358469,7.038937
1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.684860,8.135225,5.863249,1.691389,0.074312,0.000699,0.000074,0.001353,0.000296,0.000924,0.005652,0.216759,0.354225,6.938374
1707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.803270,3.639012,3.821073,1.607873,0.082323,0.000801,0.000170,0.000905,0.000163,0.001397,0.005873,0.248870,0.386573,7.001172
1708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.243704,4.766828,2.764217,1.065328,0.054239,0.000531,0.000209,0.001079,0.000207,0.001068,0.005906,0.213803,0.366559,7.096071


In [72]:
import pyedflib

with pyedflib.EdfReader('/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_BL_6h_fully_marked.edf') as edf_reader:
    n_channels = edf_reader.signals_in_file
    signal_labels = edf_reader.getSignalLabels()


    channel_index = signal_labels.index('FrL')

    data = edf_reader.readSignal(channel_index)
    sfreq = edf_reader.getSampleFrequency(channel_index)
    nyquist = 0.5 * sfreq  # Определяем nyquist здесь
    total_duration = len(data) / sfreq


start_time = 178
duration = 84

end_time = start_time + duration

if end_time > total_duration:
    end_time = total_duration
    duration = end_time - start_time

start_idx = int(start_time * sfreq)
end_idx = int(end_time * sfreq)

signal_data = data[start_idx:end_idx]

features = extract_features(signal_data, 256)

df_output = pd.DataFrame(features, columns=["Feature_Value"])
df_output.transpose()

6 4 3 1


/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Feature_Value,469.812174,163.132219,113.45102,26.025194,2.526445,0.037146,0.010236,0.004779,0.003106,0.001008,0.023057,0.153719,0.382257,6.761762


In [74]:
edf_path = '/Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)/Ati4x1_15m_BL_6h_fully_marked.edf'

# Load the EDF file
with pyedflib.EdfReader(edf_path) as edf_reader:
    n_channels = edf_reader.signals_in_file
    signal_labels = edf_reader.getSignalLabels()

    # Specify the desired channel
    channel_index = signal_labels.index('FrL')

    # Load the signal and its sample frequency
    data = edf_reader.readSignal(channel_index)
    sfreq = edf_reader.getSampleFrequency(channel_index)
    nyquist = 0.5 * sfreq
    total_duration = len(data) / sfreq

# List to hold feature rows
feature_rows = []

# Iterate through each row in the dataset
for _, row in all_valid_intervals.iterrows():
    start_time = row['X_FROM']
    duration = row['X_TO'] - row['X_FROM']
    end_time = start_time + duration

    # Adjust `end_time` if it exceeds the total signal duration
    if end_time > total_duration:
        end_time = total_duration
        duration = end_time - start_time

    # Convert times to indices
    start_idx = int(start_time * sfreq)
    end_idx = int(end_time * sfreq)

    # Extract the signal segment
    signal_data = data[start_idx:end_idx]

    # Extract features from this segment
    features = extract_features(signal_data, sfreq)

    # Convert features to DataFrame row and transpose
    feature_row = pd.DataFrame([features]).T
    feature_row = feature_row.transpose()
    feature_row.columns = [f"Feature_{i+1}" for i in range(len(features))]

    # Append the feature row
    feature_rows.append(feature_row)

# Concatenate all feature rows with the original DataFrame
features_df = pd.concat(feature_rows, ignore_index=True)
df_with_features = pd.concat([all_valid_intervals.reset_index(drop=True), features_df], axis=1)
df_with_features

/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1


/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1


/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

...

6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1


/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1
6 4 3 1


/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:36: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  theta_power = np.trapz(psd[(freqs >= 4) & (freqs <= 8)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  alpha_power = np.trapz(psd[(freqs >= 8) & (freqs <= 13)])
/var/folders/b6/hs82fhv10h35l3krx6tyhln00000gq/T/ipykernel_73629/2264793073.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of th

,X_FROM,X_TO,LABEL,ID,AGE_MONTHS,EXPERIMENT_TYPE,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14
0,178.0,262.0,ds,Ati4x1,15m,BL,497.139988,163.449072,113.678837,15.514513,0.423272,0.003260,0.003017,0.002368,0.001273,0.001656,0.023515,0.132527,0.272328,6.885228
1,343.0,496.0,ds,Ati4x1,15m,BL,556.769463,211.224732,130.742116,20.613215,0.611436,0.004898,0.001687,0.001797,0.000893,0.001101,0.015027,0.136749,0.281314,6.710697
2,1475.0,1501.0,ds,Ati4x1,15m,BL,131.094962,69.272630,40.533135,6.670344,0.186059,0.001444,0.002254,0.003267,0.001630,0.002287,0.023784,0.149181,0.280805,6.714052
3,1511.0,1772.0,ds,Ati4x1,15m,BL,1172.696641,475.733809,330.933096,46.621178,1.270823,0.009873,0.002154,0.002210,0.001223,0.001636,0.019415,0.141456,0.275315,6.578759
4,1774.0,1791.0,is,Ati4x1,15m,BL,54.161166,65.167860,36.579662,4.998324,0.126973,0.000974,0.001250,0.003307,0.003210,0.002552,0.023619,0.168110,0.266278,7.247415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,21490.0,21528.0,swd,Ati4x3,9m,Xyl01(Pharm!),79.243311,54.739345,28.922468,6.221041,0.188316,0.001486,0.000911,0.001932,0.000714,0.001097,0.011119,0.158714,0.302842,6.764228
566,21531.0,21540.0,swd,Ati4x3,9m,Xyl01(Pharm!),14.016734,7.279730,6.055246,1.415562,0.043864,0.000339,0.000630,0.000981,0.000605,0.001021,0.007858,0.179400,0.317639,7.089789
567,21547.0,21548.0,swd,Ati4x3,9m,Xyl01(Pharm!),1.126257,0.194950,0.241914,0.116000,0.003755,0.000032,0.000868,0.000207,0.000111,0.000509,0.004016,0.170039,0.365733,7.069731
568,21554.0,21587.0,swd,Ati4x3,9m,Xyl01(Pharm!),38.956355,13.621938,10.920263,3.907106,0.137449,0.001196,0.000564,0.000437,0.000267,0.000521,0.005044,0.167655,0.348154,6.864988


In [64]:
signal_data.shape

(33600,)

In [50]:
signal_data

array([-0.167625 , -0.1756875, -0.1766875, ..., -0.0583125, -0.0336875,
       -0.0354375])

In [44]:
signal_labels

['FrL', 'FrR', 'OcR']

In [42]:
data.shape

(8640400,)

In [43]:
sfreq

400.0

In [36]:
all_valid_intervals.shape

(570, 6)

In [38]:
all_valid_intervals.LABEL.value_counts()

LABEL
swd    363
ds     169
is      38
Name: count, dtype: int64

In [10]:
valid_intervals.LABEL.value_counts()

LABEL
ds    57
is    11
Name: count, dtype: int64

In [ ]:
# /Users/s.vandanov/Documents/GitHub/classification-system-ekog/data/ECoG_fully_marked_(4+2 files, 6 h each)
